In [2]:
import os
RMG_PY_PATH = os.path.expanduser('/gss_gpfs_scratch/westgroup/Importer/RMG-Py/')
RMG_MODELS_PATH = os.path.expanduser('/gss_gpfs_scratch/westgroup/Importer/RMG-models')

import sys
import re

sys.path.insert(1,RMG_PY_PATH)
from rmgpy.molecule import Molecule
import rmgpy.kinetics
import rmgpy.rmg
import rmgpy.data
import rmgpy.data.kinetics
import numpy
import cPickle as pickle
from collections import Counter, defaultdict
from rmgpy.reaction import Reaction
import pandas as pd
import ck2cti
import cantera as ct
import itertools
import random
import numpy as np
import itertools




In [3]:
# Getting the OOH reactions
f = open("../../autotst_kinetics.pkl","r")
autotst_kinetics = pickle.load(f)
ooh_reactions = []
for rxn in autotst_kinetics:
    reactants, products = rxn.label.split("_")
    r1, r2 = reactants.split('+')
    p1, p2 = products.split('+')
    if "OO" in [r1, r2, p1, p2] and "[O]O" in [r1, r2, p1, p2]:
        #print [r1, r2, p1, p2]
        ooh_reactions.append(rxn)

In [4]:
master = 'PCI2013/335-Wang/'

# Find and read the chemkin file
with open(os.path.join(RMG_MODELS_PATH, master,'import.sh')) as infile:
    shellscript = infile.read()
reactions_filename = re.search('--reactions\s+(\S+)',shellscript).group(1)
reactions_filepath = os.path.join(RMG_MODELS_PATH,master,reactions_filename)
thermo_filename = re.search('--thermo\s+(\S+)',shellscript).group(1)
thermo_filepath = os.path.join(RMG_MODELS_PATH,master,thermo_filename)
print(reactions_filepath)
print(thermo_filepath)
with open(reactions_filepath) as infile:
    chemkin = infile.readlines()
print "".join(chemkin[:4]) # print first 4 lines only

parser = ck2cti.Parser()
surfaces = parser.convertMech(inputFile=reactions_filepath,
                              thermoFile=thermo_filepath,
                              transportFile=None,
                              surfaceFile=None,
                              phaseName=None,
                              outName='master_wang.cti',
                              permissive=True)

/gss_gpfs_scratch/westgroup/Importer/RMG-models/PCI2013/335-Wang/mechanism.txt
/gss_gpfs_scratch/westgroup/Importer/RMG-models/PCI2013/335-Wang/thermo.txt
!References:
!
!A shock tube, rapid compression machine, and kinetic modeling study of the autoignition of 3-methylheptane
!W. Wang, Z. Li, M.A. Oehlschlaeger, D. Healy, H.J. Curran, S.M. Sarathy, W.J. Pitz, M. Mehl, C.K. Westbrook

Wrote CTI mechanism file to 'master_wang.cti'.
Mechanism contains 1378 species and 8143 reactions.


In [5]:
dict_path = os.path.join(RMG_MODELS_PATH, master, 'RMG-Py-kinetics-library', 'dictionary.txt')
print "Loading species_dict from",dict_path
species_dict = rmgpy.data.kinetics.KineticsLibrary().getSpecies(dict_path)

Loading species_dict from /gss_gpfs_scratch/westgroup/Importer/RMG-models/PCI2013/335-Wang/RMG-Py-kinetics-library/dictionary.txt


In [6]:
species_dict

{'ic4h7o': Species(label="ic4h7o", molecule=[Molecule(SMILES="C=C(C)C[O]")]),
 'hoch2oco': Species(label="hoch2oco", molecule=[Molecule(SMILES="O=[C]OCO")]),
 'c7h14ooh3-5o2': Species(label="c7h14ooh3-5o2", molecule=[Molecule(SMILES="CCC(CC(CC)OO)O[O]")]),
 'ch3chcho': Species(label="ch3chcho", molecule=[Molecule(SMILES="C[CH]C=O"), Molecule(SMILES="CC=C[O]")]),
 'c8h17ooh-1-2': Species(label="c8h17ooh-1-2", molecule=[Molecule(SMILES="CCCCCC(C)COO")]),
 'ic3h7coc3h6p': Species(label="ic3h7coc3h6p", molecule=[Molecule(SMILES="[CH2]CCC(=O)C(C)C")]),
 'tc4h8ooh-io2': Species(label="tc4h8ooh-io2", molecule=[Molecule(SMILES="CC(C)(CO[O])OO")]),
 'c8coc2h4p': Species(label="c8coc2h4p", molecule=[Molecule(SMILES="[CH2]CC(=O)CCCCCCCC")]),
 'nc5ket32': Species(label="nc5ket32", molecule=[Molecule(SMILES="CCC(=O)C(C)OO")]),
 'nc4h9coch2': Species(label="nc4h9coch2", molecule=[Molecule(SMILES="[CH2]C(=O)CCCC"), Molecule(SMILES="C=C([O])CCCC")]),
 'ch3och2o2': Species(label="ch3och2o2", molecule=[

In [7]:
smiles_to_nickname_dict = {}
for species in species_dict.itervalues():
    for mol in species.molecule:
        smiles_to_nickname_dict[mol.toSMILES()] = species.label
smiles_to_nickname_dict

{'CC(CCC(C)C(C)OO)O[O]': 'c8ooh2-oo6-3',
 'CC(C)(C=O)CO[O]': 'o2c4h8cho',
 'CCCC(=O)CCOO': 'nc6ket31',
 'CCC(C)C(CC(C)=O)OO': 'c8ket6-4-3',
 'CC[C]=O': 'c2h5co',
 'CCC=C[CH]COO': 'c6h103ooh1-2',
 'CC(C)CC[CH]C(C)OO': 'c8ooh6-2e',
 'CCC(C)CC([O])CC': 'c8h17o-5-3',
 '[CH2]CC=CCCC': 'c7h133-1',
 'CCC(C)C(OO)C(CC)O[O]': 'c8ooh4-oo5-3',
 '[CH2]CCCCC=O': 'c5h10cho-1',
 'CCCCC[C](C)COO': 'c8ooh1-2b',
 'CCC(=O)CC(C)OO': 'nc6ket35',
 'CCCC(=O)C(OO)C(C)C': 'c8ket4-3-2',
 '[CH2]CCCCC=C': 'c7h131-7',
 'C[CH]C(C)C(CCC)OO': 'c8ooh4-3b',
 'C[CH]C=CCC': 'c6h112-4',
 'C=C([O])C(C)CCC(C)C': 'ic7h15coch2',
 'C=C[CH]CC': 'c5h91-3',
 'CC1CCC(C)C(C)O1': 'c8h16o2-6-3',
 'CCCCC(OO)[C](C)C': 'c8ooh3-2b',
 'C[CH]C(C)O': 'sc4h8oh',
 'C[CH]C(C)CC': 'mc6h13',
 'CCC1OC1CC(C)C': 'c8h16o4-5-2',
 'C[CH]C(C)C': 'cc5h11',
 '[CH2]C=CC': 'c4h71-3',
 'CC=CC(CC)O[O]': 'c6h112o2-4',
 'CCC(C)CC(=O)C(C)OO': 'c8ket5-6-3',
 'C[CH]C(CC(C)CC)OO': 'c8ooh5-3f',
 '[CH2]CCC(C)C': 'ec6h13',
 'CCC(OO)C(C)=O': 'nc5ket23',
 'CCOO': 'c2h5o

In [8]:
inchikey_to_smiles_dict = {}
for smiles in smiles_to_nickname_dict.iterkeys():
    inchikey_to_smiles_dict[Molecule(SMILES=smiles).toInChIKey()] = smiles
    
inchikey_to_smiles_dict

{'RYKZRKKEYSRDNF-UHFFFAOYSA': 'C=C(CC)CC',
 'LTSFJKSLJFUCFW-UHFFFAOYSA': 'CC(C)CCCCCOO',
 'IISRDCOVUDUNJX-UHFFFAOYSA': 'CC=CC(CCOO)O[O]',
 'SOMGOFGLBLEIAO-UHFFFAOYSA': 'CC(CCC(C)O[O])COO',
 'MWKIYYPCAHOAFS-UHFFFAOYSA': 'CCC(=O)CCCCOO',
 'ZXJXRBXBUKJGCQ-UHFFFAOYSA': 'CCCC(CCCCO[O])OO',
 'XVBSQDRVAFVNPI-UHFFFAOYSA': 'CC(C=O)CCC(C)OO',
 'AKJQDAQYJFNQGD-UHFFFAOYSA': 'C[CH]CCC(C)COO',
 'OMJIBOMAQJNQHG-UHFFFAOYSA': '[CH2]CCC(C=C)OO',
 'YWFLJXPALMDRIV-UHFFFAOYSA': 'CC(C)CCC1CCO1',
 'PZJFUXNCZMWYQR-UHFFFAOYSA': 'CC(CO[O])COO',
 'PPFXLEQGLWTURZ-UHFFFAOYSA': '[CH2]CCCC=C',
 'BVUKVKWSSJYLQQ-UHFFFAOYSA': 'C=CCCCOO',
 'WBTDTLOHMGSJMJ-UHFFFAOYSA': 'CC(C)CCCCCO[O]',
 'LBXWSYGTQCAJSK-UHFFFAOYSA': 'CC=CC([O])CC',
 'PMETXYSKYFVPEC-UHFFFAOYSA': 'C=CC[CH]CCCC',
 'HAAHANSKHZOLBU-UHFFFAOYSA': 'C=CC=C[O]',
 'NAMZKTVBCXXPQA-UHFFFAOYSA': 'CC(C)CCCCC[O]',
 'KFJTVXLXQOYPGN-UHFFFAOYSA': 'CCCC(CCCO[O])OO',
 'BJMLBWZYFZYLNJ-UHFFFAOYSA': 'CCCCCC(O[O])C(C)OO',
 'TWBIDQDUOXHFSN-UHFFFAOYSA': 'CC=C[O]',
 'SCLQNSGCBOMUOJ

In [9]:
"h2o2" in species_dict.iterkeys()

True

In [10]:
sarathy_rxns = []
for r in parser.reactions:

    split_string = str(r).split()
    #print split_string
    if not (smiles_to_nickname_dict["OO"] in split_string and smiles_to_nickname_dict["[O]O"] in split_string):
        #print "boo"
        continue

    elif (len(split_string) == 7 and
        split_string[0] in species_dict.iterkeys() and
        split_string[2] in species_dict.iterkeys() and
        split_string[4] in species_dict.iterkeys() and
        split_string[6] in species_dict.iterkeys()):
        print str(r)

        # Creating a list of possible reaction labels
        reactants = [species_dict[n].molecule[0].toSMILES() for n in r.reactantString.split(' + ')]
        products = [species_dict[n].molecule[0].toSMILES() for n in r.productString.split(' + ')]
        joined_reactant_orders = ['+'.join(order) for order in itertools.permutations(reactants)]
        joined_product_orders = ['+'.join(order) for order in itertools.permutations(products)]
        possible_labels = ['_'.join((joined_r, joined_p)) for joined_r in joined_reactant_orders for joined_p in joined_product_orders]

        for reaction in ooh_reactions:
            if not reaction.label in possible_labels:
                continue
                
            got_r = False
            got_p = False
            print reaction

            try:
                for reactant in reaction.reactants:
                    inchi_key = reactant.label.split("-u")[0]

                    reactant.label = smiles_to_nickname_dict[inchikey_to_smiles_dict[inchi_key]]
                    reactant.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
                got_r = True
                print "got r"

            except KeyError:
                #print "Key Error with reactants... skipping"
                continue

            try:
                for product in reaction.products:
                    inchi_key = product.label.split("-u")[0]


                    product.label = smiles_to_nickname_dict[inchikey_to_smiles_dict[inchi_key]]
                    product.molecule = [Molecule(SMILES=inchikey_to_smiles_dict[inchi_key])]
                got_p = True
                print "got p"
            except KeyError:
                #print "Key Error with products... skipping"
                continue




            # Then append the reaction to our reaction list
            if got_p == True and got_r == True:
                print 
                print r.reactantString
                print

                sarathy_rxns.append([r, reaction, reaction.toChemkin(), reaction.toCantera()])
#print sarathy_rxns

h2o2 + o2 -> ho2 + ho2
ho2 + ho2 -> h2o2 + o2
h2o2 + o2 -> ho2 + ho2
ho2 + ho2 -> h2o2 + o2
h2o2 + h -> h2 + ho2
MHAJPDPJQMAIIY-UHFFFAOYSA + YZCKVEUIGOORGS-UHFFFAOYSA-u1 <=> UFHFLCQGNIYNRP-UHFFFAOYSA + OUUQCZGPVNCOIJ-UHFFFAOYSA-u2
got r
got p

h2o2 + h

h2 + ho2 -> h2o2 + h
h2o2 + o -> oh + ho2
oh + ho2 -> h2o2 + o
h2o2 + oh -> h2o + ho2
h2o + ho2 -> h2o2 + oh
h2o2 + oh -> h2o + ho2
h2o + ho2 -> h2o2 + oh
ch2o + ho2 -> hco + h2o2
WSFSSNUMVMOOMR-UHFFFAOYSA + OUUQCZGPVNCOIJ-UHFFFAOYSA-u2 <=> MHAJPDPJQMAIIY-UHFFFAOYSA + CFHIDWOYWUOIHU-UHFFFAOYSA-u1
got r
got p

ch2o + ho2

hco + h2o2 -> ch2o + ho2
CFHIDWOYWUOIHU-UHFFFAOYSA-u1 + MHAJPDPJQMAIIY-UHFFFAOYSA <=> WSFSSNUMVMOOMR-UHFFFAOYSA + OUUQCZGPVNCOIJ-UHFFFAOYSA-u2
got r
got p

hco + h2o2

hocho + ho2 -> ocho + h2o2
OUUQCZGPVNCOIJ-UHFFFAOYSA-u2 + BDAGIHXWWSANSR-UHFFFAOYSA <=> JRCLCXAVSVCEQC-UHFFFAOYSA-u2 + MHAJPDPJQMAIIY-UHFFFAOYSA
got r
got p

hocho + ho2

ocho + h2o2 -> hocho + ho2
ch3o + ho2 -> ch2o + h2o2
ch2o + h2o2 -> ch3o + ho2
ch2oh

In [11]:
wang_df = pd.DataFrame(sarathy_rxns)
wang_df.columns = ["Wang Reaction", "AutoTST Reaction", "AutoTST Reaction - Chemkin" , "AutoTST - Cantera"]
wang_df

,Wang Reaction,AutoTST Reaction,AutoTST Reaction - Chemkin,AutoTST - Cantera
0,h2o2 + h -> h2 + ho2,h2o2 + h <=> h2 + ho2,"! Fitted to 59 data points; dA = *|/ 1.57377, ...",h + h2o2 <=> h2 + ho2
1,ch2o + ho2 -> hco + h2o2,ch2o + ho2 <=> h2o2 + hco,"! Fitted to 59 data points; dA = *|/ 2.04573, ...",ch2o + ho2 <=> h2o2 + hco
2,hco + h2o2 -> ch2o + ho2,hco + h2o2 <=> ch2o + ho2,"! Fitted to 59 data points; dA = *|/ 1.80424, ...",h2o2 + hco <=> ch2o + ho2
3,hocho + ho2 -> ocho + h2o2,ho2 + hocho <=> ocho + h2o2,"! Fitted to 59 data points; dA = *|/ 1.02165, ...",ho2 + hocho <=> h2o2 + ocho
4,ch3oh + ho2 -> ch2oh + h2o2,ho2 + ch3oh <=> ch2oh + h2o2,"! Fitted to 59 data points; dA = *|/ 1.49501, ...",ch3oh + ho2 <=> ch2oh + h2o2
5,ch2oh + h2o2 -> ch3oh + ho2,ch2oh + h2o2 <=> ch3oh + ho2,"! Fitted to 59 data points; dA = *|/ 1.11474, ...",ch2oh + h2o2 <=> ch3oh + ho2
6,c2h6 + ho2 -> c2h5 + h2o2,c2h6 + ho2 <=> c2h5 + h2o2,"! Fitted to 59 data points; dA = *|/ 1.37627, ...",c2h6 + ho2 <=> c2h5 + h2o2
7,c2h5 + h2o2 -> c2h6 + ho2,c2h5 + h2o2 <=> c2h6 + ho2,"! Fitted to 59 data points; dA = *|/ 1.12702, ...",c2h5 + h2o2 <=> c2h6 + ho2
8,c2h4o1-2 + ho2 -> c2h3o1-2 + h2o2,c2h4o1-2 + ho2 <=> h2o2 + c2h3o1-2,"! Fitted to 59 data points; dA = *|/ 1.07588, ...",c2h4o1-2 + ho2 <=> c2h3o1-2 + h2o2
9,ch3cho + ho2 -> ch3co + h2o2,ch3cho + ho2 <=> h2o2 + ch3co,"! Fitted to 59 data points; dA = *|/ 1.59886, ...",ch3cho + ho2 <=> ch3co + h2o2


In [12]:
stored_r = []
for r in parser.reactions:
    split_string = str(r).split()
    split_string
    if not (smiles_to_nickname_dict["OO"] in split_string and smiles_to_nickname_dict["[O]O"] in split_string):
        #print "boo"
        continue
        
    elif (len(split_string) == 7 and
        split_string[0] in species_dict.iterkeys() and 
        split_string[2] in species_dict.iterkeys() and
        split_string[4] in species_dict.iterkeys() and 
        split_string[6] in species_dict.iterkeys()):
        print str(r)
        
        stored_r.append(r)
        
        
stored_r        

h2o2 + o2 -> ho2 + ho2
ho2 + ho2 -> h2o2 + o2
h2o2 + o2 -> ho2 + ho2
ho2 + ho2 -> h2o2 + o2
h2o2 + h -> h2 + ho2
h2 + ho2 -> h2o2 + h
h2o2 + o -> oh + ho2
oh + ho2 -> h2o2 + o
h2o2 + oh -> h2o + ho2
h2o + ho2 -> h2o2 + oh
h2o2 + oh -> h2o + ho2
h2o + ho2 -> h2o2 + oh
ch2o + ho2 -> hco + h2o2
hco + h2o2 -> ch2o + ho2
hocho + ho2 -> ocho + h2o2
ocho + h2o2 -> hocho + ho2
ch3o + ho2 -> ch2o + h2o2
ch2o + h2o2 -> ch3o + ho2
ch2oh + ho2 -> ch2o + h2o2
ch2o + h2o2 -> ch2oh + ho2
ch3oh + ho2 -> ch2oh + h2o2
ch2oh + h2o2 -> ch3oh + ho2
ch4 + ho2 -> ch3 + h2o2
ch3 + h2o2 -> ch4 + ho2
c2h6 + ho2 -> c2h5 + h2o2
c2h5 + h2o2 -> c2h6 + ho2
c2h4o1-2 + ho2 -> c2h3o1-2 + h2o2
c2h3o1-2 + h2o2 -> c2h4o1-2 + ho2
ch3cho + ho2 -> ch3co + h2o2
ch3co + h2o2 -> ch3cho + ho2
h2o2 + ch3co3 -> ho2 + ch3co3h
ho2 + ch3co3h -> h2o2 + ch3co3
c2h5oh + ho2 -> pc2h4oh + h2o2
pc2h4oh + h2o2 -> c2h5oh + ho2
c2h5oh + ho2 -> sc2h4oh + h2o2
sc2h4oh + h2o2 -> c2h5oh + ho2
c2h5oh + ho2 -> c2h5o + h2o2
c2h5o + h2o2 -> c2h5oh + 

In [13]:
for r in stored_r:
    reactants = [species_dict[n].molecule[0].toSMILES() for n in r.reactantString.split(' + ')]
    products = [species_dict[n].molecule[0].toSMILES() for n in r.productString.split(' + ')]
    joined_reactant_orders = ['+'.join(order) for order in itertools.permutations(reactants)]
    joined_product_orders = ['+'.join(order) for order in itertools.permutations(products)]
    possible_labels = ['_'.join((joined_r, joined_p)) for joined_r in joined_reactant_orders for joined_p in joined_product_orders]
    for reaction in ooh_reactions:
        #print reaction.label
        if reaction.label in possible_labels:
            print reaction.label
            rreaction = reaction
        
        

OO+[H]_[H][H]+[O]O
C=O+[O]O_OO+[CH]=O
OO+[CH]=O_C=O+[O]O
O=CO+[O]O_OO+[O]C=O
CO+[O]O_OO+[CH2]O
OO+[CH2]O_CO+[O]O
CC+[O]O_C[CH2]+OO
C[CH2]+OO_CC+[O]O
C1CO1+[O]O_OO+[CH]1CO1
CC=O+[O]O_C[C]=O+OO
C[C]=O+OO_CC=O+[O]O
CC(=O)O[O]+OO_CC(=O)OO+[O]O
CCO+[O]O_OO+[CH2]CO
CCO+[O]O_C[CH]O+OO
OO+[CH2]C(C)=O_CC(C)=O+[O]O
C=CC=O+[O]O_C=C[C]=O+OO
CCC=O+[O]O_CC[C]=O+OO
COC+[O]O_OO+[CH2]OC
COC=O+[O]O_CO[C]=O+OO
COC=O+[O]O_OO+[CH2]OC=O
CCC+[O]O_C[CH]C+OO
CCC+[O]O_OO+[CH2]CC
OO+[CH2]CC_CCC+[O]O
C=CC+[O]O_OO+[CH2]C=C
OO+[CH2]C=C_C=CC+[O]O
CCCC+[O]O_OO+[CH2]CCC
CCCC+[O]O_C[CH]CC+OO
C=C[CH]C+OO_C=CCC+[O]O
C=CCC+[O]O_OO+[CH2]CC=C
CCC(C)O[O]+OO_CCC(C)OO+[O]O
CCC(C)=O+[O]O_OO+[CH2]CC(C)=O
CCC(C)=O+[O]O_OO+[CH2]C(=O)CC
CCCC=O+[O]O_CCC[C]=O+OO
CCCC=O+[O]O_OO+[CH2]CCC=O
CCCC=O+[O]O_C[CH]CC=O+OO
CCCC=O+[O]O_CC[CH]C=O+OO
CC=CC=O+[O]O_CC=C[C]=O+OO
CC(C)C+[O]O_OO+[CH2]C(C)C
OO+[CH2]C(C)C_CC(C)C+[O]O
CC(C)CO[O]+OO_CC(C)COO+[O]O
CC(C)(C)O[O]+OO_CC(C)(C)OO+[O]O
CC(C)C=O+[O]O_CC(C)[C]=O+OO
CC(C)C=O+[O]O_C[C](C)C=O+OO
OO+[CH

In [15]:
parser.energy_units = 'kcal/mol'
reactions_by_cti_string = {}
for entry in zip(wang_df.iloc[:,1], wang_df.iloc[:,2]):
    reaction, chemkin_string = entry
    kinetics = reaction.kinetics
    chemkin_string = '\n'.join([l for l in chemkin_string.splitlines() if not l.startswith('!')])
    chemkin_string
    cantera_reaction, reverse_reaction = parser.readKineticsEntry(chemkin_string, False)
    cti_string = str(cantera_reaction)
    reactions_by_cti_string[cti_string] = (reaction, chemkin_string, cantera_reaction)

print len(reactions_by_cti_string)
print '\n'.join(sorted(reactions_by_cti_string.keys()))

144
ac3h5cho + ho2 <=> ac3h5co + h2o2
ac3h5co + h2o2 <=> ac3h5cho + ho2
c2h3cho + ho2 <=> c2h3co + h2o2
c2h4o1-2 + ho2 <=> h2o2 + c2h3o1-2
c2h5 + h2o2 <=> c2h6 + ho2
c2h5cho + ho2 <=> c2h5co + h2o2
c2h5coc2h3 + ho2 <=> h2o2 + pc2h4coc2h3
c2h5coc2h3 + ho2 <=> h2o2 + sc2h4coc2h3
c2h5coc2h5 + ho2 <=> c2h5coc2h4p + h2o2
c2h5coch3 + ho2 <=> c2h5coch2 + h2o2
c2h5oh + ho2 <=> pc2h4oh + h2o2
c2h6 + ho2 <=> c2h5 + h2o2
c3h5-a + h2o2 <=> c3h6 + ho2
c3h6 + ho2 <=> h2o2 + c3h5-a
c3h8 + ho2 <=> h2o2 + ic3h7
c3h8 + ho2 <=> h2o2 + nc3h7
c4h10 + ho2 <=> pc4h9 + h2o2
c4h6cho1-43 + h2o2 <=> c4h7cho1-4 + ho2
c4h7cho1-4 + ho2 <=> h2o2 + c4h7co1-4
c4h8coch3-2 + h2o2 <=> nc4h9coch3 + ho2
c4h8coch3-3 + h2o2 <=> nc4h9coch3 + ho2
c5h10-1 + ho2 <=> c5h91-5 + h2o2
c5h10-2 + ho2 <=> h2o2 + c5h91-3
c5h10-2 + ho2 <=> h2o2 + c5h92-5
c5h10cho-1 + h2o2 <=> ho2 + nc5h11cho
c5h10cho-2 + h2o2 <=> ho2 + nc5h11cho
c5h10cho-3 + h2o2 <=> nc5h11cho + ho2
c5h10cho-4 + h2o2 <=> ho2 + nc5h11cho
c5h11-2 + h2o2 <=> nc5h12 + ho2
c5

In [17]:
import textwrap
original_reactions = list()
alternatives_rates = defaultdict(list)
for i, r in enumerate(parser.reactions):
    original_reactions.append(r)
    assert original_reactions[i] == parser.reactions[i]
    #print list(wang_df.iloc[:,0])


    if r in list(wang_df.iloc[:,0]):
        _, rmg_reaction, kinetics_options, ct_version_of_rmg_kinetics = wang_df[r == wang_df.iloc[:,0]].values[0]
        # we just want the rmg_reaction


        chemkin_string = rmg_reaction.toChemkin()
        chemkin_string = '\n'.join([l for l in chemkin_string.splitlines() if not l.startswith('!')])
        r2, reverse_reaction = parser.readKineticsEntry(chemkin_string, False)
        # r2 is the new replacement reaction

        r2.comment = r.comment
        r2.comment += '\n \nSUBSTITUTION: The following reaction was originally\n{}\n'.format(r.to_cti())
        r2.comment += 'But has been replaced with the following as seen in AutoTST'

        alternatives_rates[i].append(r2)

In [18]:
alternatives_rates

defaultdict(list,
            {32: [<ck2cti.Reaction at 0x8c3eed0>],
             85: [<ck2cti.Reaction at 0x7566690>],
             86: [<ck2cti.Reaction at 0x7566410>],
             104: [<ck2cti.Reaction at 0x7566750>],
             168: [<ck2cti.Reaction at 0x8db6810>],
             169: [<ck2cti.Reaction at 0x6328150>],
             284: [<ck2cti.Reaction at 0x7566890>],
             285: [<ck2cti.Reaction at 0xa0aecd0>],
             365: [<ck2cti.Reaction at 0xa0ae990>],
             391: [<ck2cti.Reaction at 0x8db68d0>],
             392: [<ck2cti.Reaction at 0x8db6d50>],
             412: [<ck2cti.Reaction at 0xa0aeb10>],
             533: [<ck2cti.Reaction at 0x7566f50>],
             535: [<ck2cti.Reaction at 0x7566950>],
             584: [<ck2cti.Reaction at 0xa0aee50>],
             610: [<ck2cti.Reaction at 0x8db6410>],
             632: [<ck2cti.Reaction at 0x7566610>],
             659: [<ck2cti.Reaction at 0x8d3e310>],
             732: [<ck2cti.Reaction at 0x8d3e390>

In [19]:
print "original reactions", len(parser.reactions)
print "reactions to change", len(alternatives_rates)
c = Counter(map(len,alternatives_rates.values()))
print "substitutions", sorted(c.items())
print "total variations", sum(map(len,alternatives_rates.values()))

original reactions 8143
reactions to change 144
substitutions [(1, 144)]
total variations 144


In [21]:
outdir = '../reference_files/cantera_sub_models/wang/'
os.mkdir(outdir)
for f in os.listdir(outdir):
    os.unlink(os.path.join(outdir, f))

In [22]:
parser.writeCTI(outName=os.path.join(outdir,'master.0.0.cti'))
for i, options in alternatives_rates.iteritems():
    for j, replacement in enumerate(options):
        output_filename = 'master.{}.{}.cti'.format(i+1,j+1)
        parser.reactions[i] = replacement
        ## save the file!
        header = ["####################################################",
                  "##  Reaction {:2d} has been replaced with option {:2d}  ##".format(i+1,j+1),
                  "####################################################"]
        print "saving", output_filename
        parser.writeCTI(header=header, outName=os.path.join(outdir,output_filename))
        # restore original
        parser.reactions[i] = original_reactions[i]

saving master.4098.1.cti
saving master.4100.1.cti
saving master.4102.1.cti
saving master.2056.1.cti
saving master.2058.1.cti
saving master.534.1.cti
saving master.2583.1.cti
saving master.536.1.cti
saving master.3097.1.cti
saving master.3098.1.cti
saving master.3099.1.cti
saving master.3614.1.cti
saving master.3615.1.cti
saving master.3616.1.cti
saving master.33.1.cti
saving master.3618.1.cti
saving master.3619.1.cti
saving master.3620.1.cti
saving master.3621.1.cti
saving master.7207.1.cti
saving master.7208.1.cti
saving master.7209.1.cti
saving master.7210.1.cti
saving master.4104.1.cti
saving master.3617.1.cti
saving master.8101.1.cti
saving master.2629.1.cti
saving master.2631.1.cti
saving master.2633.1.cti
saving master.2634.1.cti
saving master.86.1.cti
saving master.87.1.cti
saving master.611.1.cti
saving master.105.1.cti
saving master.633.1.cti
saving master.1814.1.cti
saving master.5314.1.cti
saving master.2187.1.cti
saving master.1816.1.cti
saving master.2189.1.cti
saving mast